In [1]:
#! pip install web3
#! pip install eth_abi

In [2]:
from web3.auto import w3
import secrets
from eth_account import Account
from eth_abi import encode_abi
from eth_abi.packed import encode_abi_packed
import web3
from hexbytes.main import HexBytes
from eth_account.messages import encode_defunct

# Get the signer address

The private key needs to be kept safe locally.
This will never ever leave your local environment.

In [3]:
private_key = secrets.token_hex(32)
private_key = "0x"+private_key
private_key

'0x6101bda470935a155228a0335f8de1a52d4a3c808087aa7b47e4b99a41350b96'

This public key sits on the contract to verify the signed message.

In [4]:
acct = Account.from_key(private_key)
public_key = acct.address
public_key

'0xb1929Cf7002Ad50c28CAeCF57cE2CaaE1758BeA4'

# Construct a ticketnumbered voucher

In [9]:
wl_address    = "0x5B38Da6a701c568545dCfcB03FcB875f56beddC4"
ticketNumbers = [1,2] 

signatures = []
for ticketNumber in ticketNumbers:
    encoded = HexBytes(w3.toHex(encode_abi_packed(['address', 'uint256'], [wl_address, ticketNumber])))
    msg = encode_defunct(hexstr=encoded.hex())
    signed = w3.eth.account.sign_message(msg, private_key=private_key)
    print("===============================")
    print("TicketNumber:", ticketNumber)
    print("messageHash", signed.messageHash.hex())
    print("signature", signed.signature.hex())
    signatures.append(signed.signature.hex())

print("===============================")
print("[\""+"\",\"".join(signatures)+"\"]")
print(ticketNumbers)

TicketNumber: 1
messageHash 0x2f5b529db9b40a80998a2da1470ac029550c30f24860621d7fdd58089d7827cd
signature 0x4d77e6b158d54d346ed3498e5dfbffce6f32c2b07936257e1503969dfe614bf665bca28d4afc999de1133942de9723451f8a6b5bd401193bf445ca25eed07fd21c
TicketNumber: 2
messageHash 0x0556b0ab30601e90d743467e40a0929dc34a5f103d14adcf79eb9ccacbb06734
signature 0x1cb460148dde3dd971142d3aeea3466272a5579b91aefe0ec55de365f376fd897533df95892304a2ea0723714d19f5016b083f9ad7737b65680e8517f6b7bf221c
["0x4d77e6b158d54d346ed3498e5dfbffce6f32c2b07936257e1503969dfe614bf665bca28d4afc999de1133942de9723451f8a6b5bd401193bf445ca25eed07fd21c","0x1cb460148dde3dd971142d3aeea3466272a5579b91aefe0ec55de365f376fd897533df95892304a2ea0723714d19f5016b083f9ad7737b65680e8517f6b7bf221c"]
[1, 2]


# Construct a message


Whitlisted address to be signed.

In [10]:
wl_address = "0xAb8483F64d9C6d1EcF9b849Ae677dD3315835cb2"

abi.encode() based encoding. Web3.py does not have this functionality for some reasons, and encode_abi from the eth_abi library needs to be used in place to generate the HexBytes object. It is not recommended to use abi.encode() on the contract side to bundle data. abi.encodePacked() is the way. 

In [11]:
binary_encoded = encode_abi(['address'], [wl_address])
encoded = HexBytes(w3.toHex(binary_encoded))
assert(encoded.hex() == "0x000000000000000000000000ab8483f64d9c6d1ecf9b849ae677dd3315835cb2")
encoded, len(encoded)

(HexBytes('0x000000000000000000000000ab8483f64d9c6d1ecf9b849ae677dd3315835cb2'),
 32)

keccak256(bytes memory). Keccak256 hashing of HexBytes if necessary.

In [12]:
keccaked1 = web3.Web3.keccak(encoded)
assert(keccaked1.hex() == "0xe92b278610ce8b6027ef7ad432103699b0132d8b129c33a54a54aa5151a105a3")
keccaked1, len(keccaked1)

(HexBytes('0xe92b278610ce8b6027ef7ad432103699b0132d8b129c33a54a54aa5151a105a3'),
 32)

keccak256(abi.encodePacked(address)).

If one wants to encode # of allowed mints, go with keccak256(abi.encodePacked(address, uint256)).


In [13]:
output = web3.Web3.solidityKeccak(['address'], [wl_address])
assert(output.hex() == "0x999bf57501565dbd2fdcea36efa2b9aef8340a8901e3459f4a4c926275d36cdb")
output, len(output)

(HexBytes('0x999bf57501565dbd2fdcea36efa2b9aef8340a8901e3459f4a4c926275d36cdb'),
 32)

ECDSA.toEthSignedMessageHash(bytes memory). 

Recreating the signed message body. This should really be done with encode_defunct() from the web3.py library.

In [14]:
output2 = web3.Web3.solidityKeccak(['string', 'string', 'bytes1'],
                                  ["\x19Ethereum Signed Message:\n", str(len(encoded)), encoded.hex()])
assert(output2.hex() == "0x0173882dc0fd8ce6d5954295f0728cf328820043b17be91b44dee6ef0c8a9eb3")
output2


HexBytes('0x0173882dc0fd8ce6d5954295f0728cf328820043b17be91b44dee6ef0c8a9eb3')

ECDSA.toEthSignedMessageHash(bytes32)

In [15]:
# ToDo. 

# Signing the message

In [16]:
msg = encode_defunct(hexstr=encoded.hex())
signed = w3.eth.account.sign_message(msg, private_key=private_key)
assert(signed.messageHash.hex() == output2.hex())
print("messageHash", signed.messageHash.hex())
print("signature", signed.signature.hex())

messageHash 0x0173882dc0fd8ce6d5954295f0728cf328820043b17be91b44dee6ef0c8a9eb3
signature 0xbd3df4655ebfee480cf9bc1204e12175bea4a818c34f3c122666c592a9c13a2326bd4d1e4a314c3d0d7fc567d49c0eaa5b33e0e83164304cd4a43ca8bdb8a0561b
